In [1]:
import os
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import pickle


In [2]:
host = os.getenv('HOST')
database = os.getenv('SQL_DATABASE')
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [3]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [4]:
engine = create_engine(connection_string)

In [5]:
insp = inspect(engine)
insp.get_table_names()

['product', 'store_cities', 'sales']

In [6]:
product_df = pd.read_sql(
    'SELECT * FROM product', engine
)
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      699 non-null    object 
 1   product_length  681 non-null    float64
 2   product_depth   683 non-null    float64
 3   product_width   683 non-null    float64
 4   cluster_id      649 non-null    object 
 5   hierarchy1_id   699 non-null    object 
 6   hierarchy2_id   699 non-null    object 
 7   hierarchy3_id   699 non-null    object 
 8   hierarchy4_id   699 non-null    object 
 9   hierarchy5_id   699 non-null    object 
dtypes: float64(3), object(7)
memory usage: 54.7+ KB


In [7]:
store_cities_df = pd.read_sql(
    'SELECT * FROM store_cities', engine
)
store_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   store_id      144 non-null    object 
 1   storetype_id  144 non-null    object 
 2   store_size    144 non-null    float64
 3   city_id       144 non-null    object 
dtypes: float64(1), object(3)
memory usage: 4.6+ KB


In [8]:
# sales_df = pd.read_sql(
#     'SELECT * FROM sales', engine
# )
# sales_df.info()

In [9]:
# filename = 'sales_df_sqlalchemy.pickle'
# with open(filename, 'wb') as f:
#     pickle.dump(sales_df, f)

In [10]:
# del sales_df

In [11]:
filename = 'sales_df_sqlalchemy.pickle'
with open(filename, 'rb') as f:
    loaded_sales_df = pickle.load(f)

In [12]:
loaded_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19454838 entries, 0 to 19454837
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   product_id             object 
 1   store_id               object 
 2   date                   object 
 3   sales                  float64
 4   revenue                float64
 5   stock                  float64
 6   price                  float64
 7   promo_type_1           object 
 8   promo_bin_1            object 
 9   promo_type_2           object 
 10  promo_bin_2            object 
 11  promo_discount_2       object 
 12  promo_discount_type_2  object 
dtypes: float64(4), object(9)
memory usage: 1.9+ GB


## Average Revenue

### 1.0 Average revenue for all the stores 

In [13]:
avg_revenue = pd.read_sql(
    'SELECT CAST(AVG(revenue) AS DECIMAL (5,2)) FROM sales', 
    engine)

avg_revenue

,avg
0,2.06


### 2.0 Average revenue for individual stores

In [14]:
avg_revenue_by_store = pd.read_sql(
    'SELECT store_id, CAST(AVG(revenue) AS DECIMAL (5,2)) FROM sales GROUP BY store_id',
    engine
)
avg_revenue_by_store

,store_id,avg
0,S0001,3.18
1,S0002,2.12
2,S0003,1.90
3,S0004,1.37
4,S0005,1.49
...,...,...
139,S0140,1.29
140,S0141,0.73
141,S0142,0.98
142,S0143,0.86


### 3.0 Average revenue for select store

### One store_id

In [15]:
store_avg_revenue = avg_revenue_by_store.loc[
    avg_revenue_by_store['store_id'] == 'S0100']
store_avg_revenue

,store_id,avg
99,S0100,2.65


### Two store_ids

In [16]:
stores = ('S0100', 'S0101')
store_avg_revenue = avg_revenue_by_store[
    avg_revenue_by_store['store_id'].isin(stores)]
store_avg_revenue

,store_id,avg
99,S0100,2.65
100,S0101,2.41


### Range of store_ids selected from table

In [17]:
select_range_store_avg_revenue = avg_revenue_by_store[
    avg_revenue_by_store['store_id'].between('S0140', 'S0144')]
select_range_store_avg_revenue

,store_id,avg
139,S0140,1.29
140,S0141,0.73
141,S0142,0.98
142,S0143,0.86
143,S0144,0.92


In [19]:
revenue_by_city_id=pd.read_sql(
'SELECT store_cities.city_id, sales.revenue FROM sales INNER JOIN store_cities ON sales.store_id=store_cities.store_id',
engine)

In [20]:
len(revenue_by_city_id['city_id'].unique())

37

In [21]:
total_revenue_by_city_id=pd.read_sql(
'SELECT city_id, CAST(SUM(revenue) AS DECIMAL (10,2)) FROM sales INNER JOIN store_cities ON sales.store_id=store_cities.store_id GROUP BY city_id',
engine
)

In [22]:
total_revenue_by_city_id

,city_id,sum
0,C001,223137.25
1,C002,680987.33
2,C003,103252.45
3,C004,1027630.08
4,C005,417360.83
5,C006,788954.70
6,C007,494121.62
7,C008,700410.20
8,C009,129320.76
9,C010,118247.03
